In [18]:
import pandas as pd
import numpy as np
import gc
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [ ]:
pd.set_option('display.max_columns', None)

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:    numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [3]:
def read_data():
    print("Reading files...")

    calendar = pd.read_csv("calendar.csv").pipe(reduce_mem_usage)
    sell_prices = pd.read_csv("sell_prices.csv").pipe(reduce_mem_usage)
    sales_train_val = pd.read_csv("sales_train_validation.csv").pipe(
        reduce_mem_usage
    )
    submission = pd.read_csv("sample_submission.csv").pipe(
        reduce_mem_usage
    )

    print("Calendar shape:", calendar.shape)
    print("Sell prices shape:", sell_prices.shape)
    print("Sales train shape:", sales_train_val.shape)
    print("Submission shape:", submission.shape)

    return calendar, sell_prices, sales_train_val, submission

In [4]:
calendar, sell_prices, sales_train_val, submission = read_data()

Reading files...
Mem. usage decreased to  0.12 Mb (0.0% reduction)
Mem. usage decreased to 130.48 Mb (23.1% reduction)
Mem. usage decreased to 95.00 Mb (0.0% reduction)
Mem. usage decreased to  2.09 Mb (0.0% reduction)
Calendar shape: (1969, 14)
Sell prices shape: (6841121, 4)
Sales train shape: (30490, 1919)
Submission shape: (60980, 29)


# Transforming sales data

In [5]:
def melt(
    sales_train_val, submission, nrows=55_000_000, verbose=True,
):
    # melt sales data, get it ready for training
    id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
    sales_train_val = pd.melt(
        sales_train_val, id_vars=id_columns, var_name="day", value_name="demand",
    )

    sales_train_val = reduce_mem_usage(sales_train_val)

    # separate test dataframes.
    test1 = submission[submission["id"].str.contains("validation")]
    # test2 = submission[submission["id"].str.contains("evaluation")]

    # change column names.
    test1.columns = ["id"] + [f"d_{x}".format(x) for x in range(1914, 1914 + 28)]
    # test2.columns = ["id"] + [f"d_{x}".format(x) for x in range(1942, 1942 + 28)]
    # get product table.
    product = sales_train_val[id_columns].drop_duplicates()

    # merge with product table
    # test2["id"] = test2["id"].str.replace("_evaluation", "_validation")
    test1 = test1.merge(product, how="left", on="id")
#     test2 = test2.merge(product, how="left", on="id")
#     test2["id"] = test2["id"].str.replace("_validation", "_evaluation")

    test1 = pd.melt(test1, id_vars=id_columns, var_name="day", value_name="demand")
    # test2 = pd.melt(test2, id_vars=id_columns, var_name="day", value_name="demand")

    sales_train_val["part"] = "train"
    test1["part"] = "test1"
    # test2["part"] = "test2"

    # data = pd.concat([sales_train_val, test1, test2], axis=0)

    data = pd.concat([sales_train_val, test1], axis=0)

    del sales_train_val, test1 # test2

    # get only a sample for fast training.
    data = data.loc[nrows:]

    # delete test2 for now.
    # data = data[data["part"] != "test2"]

    gc.collect()

    return data


def merge(data, calendar, sell_prices, verbose=True):
    # drop some calendar features.
    calendar = calendar.drop(["weekday", "wday", "month", "year"], axis=1)

    # notebook crashes with the entire dataset.
    data = pd.merge(data, calendar, how="left", left_on=["day"], right_on=["d"])
    data = data.drop(["d", "day"], axis=1)


    # get the sell price data (this feature should be very important).
    data = data.merge(sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")

    return data

In [6]:
data = melt(sales_train_val, submission, nrows=27_500_000)
data = merge(data, calendar, sell_prices)

Mem. usage decreased to 3226.27 Mb (0.0% reduction)


In [8]:
calendar.head(2)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0


In [7]:
data.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOUSEHOLD_1_514_WI_3_validation,HOUSEHOLD_1_514,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,16,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,3.500000
1,HOUSEHOLD_1_515_WI_3_validation,HOUSEHOLD_1_515,HOUSEHOLD_1,HOUSEHOLD,WI_3,WI,1,train,2013-07-18,11325,NaN,NaN,NaN,NaN,0,0,0,2.470703


In [9]:
def encode_categoricals(df):
    nan_features = ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
    for feature in nan_features:
        df[feature] = df[feature].fillna("MISSING")

    cat_cols = [
        "item_id",
        "dept_id",
        "cat_id",
        "store_id",
        "state_id",
        "event_name_1",
        "event_type_1",
        "event_name_2",
        "event_type_2",
    ]

    for col in cat_cols:
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col])

    return df

def add_agg_features(df):
    # rolling demand features.
    for shift in [28, 29, 30]:
        df[f"shift_t{shift}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(shift)
        )

    for size in [7, 30]:
        df[f"rolling_std_t{size}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(28).rolling(size).std()
        )

    for size in [7, 30, 90, 180]:
        df[f"rolling_mean_t{size}"] = df.groupby(["id"])["demand"].transform(
            lambda x: x.shift(28).rolling(size).mean()
        )

    df["rolling_skew_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(28).rolling(30).skew()
    )
    df["rolling_kurt_t30"] = df.groupby(["id"])["demand"].transform(
        lambda x: x.shift(28).rolling(30).kurt()
    )
    # price features
    df["shift_price_t1"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1)
    )
    df["price_change_t1"] = (df["shift_price_t1"] - df["sell_price"]) / (
        df["shift_price_t1"]
    )
    df["rolling_price_max_t365"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.shift(1).rolling(365).max()
    )
    df["price_change_t365"] = (df["rolling_price_max_t365"] - df["sell_price"]) / (
        df["rolling_price_max_t365"]
    )

    df["rolling_price_std_t7"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(7).std()
    )
    df["rolling_price_std_t30"] = df.groupby(["id"])["sell_price"].transform(
        lambda x: x.rolling(30).std()
    )
    return df.drop(["rolling_price_max_t365", "shift_price_t1"], axis=1)


In [10]:
def add_time_features(df):
    df["date"] = pd.to_datetime(df["date"])
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["week"] = df["date"].dt.week
    df["day"] = df["date"].dt.day
    df["dayofweek"] = df["date"].dt.dayofweek
    df["is_weekend"] = df["dayofweek"].isin([5, 6])
    return df

In [13]:
data = encode_categoricals(data)
data = add_agg_features(data)
data = add_time_features(data)
data = reduce_mem_usage(data)

Mem. usage decreased to 2658.78 Mb (42.9% reduction)


In [16]:
data.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,demand,part,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_std_t7,rolling_std_t30,rolling_mean_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_skew_t30,rolling_kurt_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,year,month,week,day,dayofweek,is_weekend
0,HOUSEHOLD_1_514_WI_3_validation,2506,5,2,9,2,16,train,2013-07-18,11325,13,1,1,1,0,0,0,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3,False
1,HOUSEHOLD_1_515_WI_3_validation,2507,5,2,9,2,1,train,2013-07-18,11325,13,1,1,1,0,0,0,2.470703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3,False


In [17]:
def train_lgb(bst_params, fit_params, X, y, cv):
    models = []

    for idx_fold, (idx_trn, idx_val) in enumerate(cv.split(X, y)):
        print(f"\n---------- Fold: ({idx_fold + 1} / {cv.get_n_splits()}) ----------\n")

        X_trn, X_val = X.iloc[idx_trn], X.iloc[idx_val]
        y_trn, y_val = y.iloc[idx_trn], y.iloc[idx_val]
        train_set = lgb.Dataset(X_trn, label=y_trn)
        val_set = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(
            bst_params,
            train_set,
            valid_sets=[train_set, val_set],
            valid_names=["train", "valid"],
            **fit_params,
        )
        models.append(model)

    return models

In [19]:
features = [
    "item_id",
    "dept_id",
    "cat_id",
    "store_id",
    "state_id",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "snap_CA",
    "snap_TX",
    "snap_WI",
    "sell_price",
    # aggregation features.
    "shift_t28",
    "shift_t29",
    "shift_t30",
    "rolling_mean_t7",
    "rolling_std_t7",
    "rolling_mean_t30",
    "rolling_mean_t90",
    "rolling_mean_t180",
    "rolling_std_t30",
    "price_change_t1",
    "price_change_t365",
    "rolling_price_std_t7",
    "rolling_price_std_t30",
    "rolling_skew_t30",
    "rolling_kurt_t30",
    # time features.
    "year",
    "month",
    "week",
    "day",
    "dayofweek",
]
# prepare training and test data.
# 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
# 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
# 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

mask1 = data["date"] <= "2016-03-27"  # noqa
mask2 = data["date"] <= "2016-04-24"  # noqa
X_train = data[mask2][features]
y_train = data[mask2]["demand"]

# X_val = data[~mask1 & mask2][features]
# y_val = data[~mask1 & mask2]["demand"]

X_test = data[~mask2][features]
id_date = data[~mask2][["id", "date"]]  # keep these two columns to use later.

del data
gc.collect()



70

In [20]:
bst_params = {
    "boosting_type": "gbdt",
    "metric": "rmse",
    "objective": "regression",
    "n_jobs": -1,
    "seed": 42,
    "learning_rate": 0.1,
    "bagging_fraction": 0.75,
    "bagging_freq": 10,
    "colsample_bytree": 0.75,
}

fit_params = {
    "num_boost_round": 100_000,
    "early_stopping_rounds": 50,
    "verbose_eval": 100,
}

cv = TimeSeriesSplit(n_splits=5)
models = train_lgb(bst_params, fit_params, X_train, y_train, cv)



---------- Fold: (1 / 5) ----------

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.73386	valid's rmse: 2.68226
Early stopping, best iteration is:
[71]	train's rmse: 2.78509	valid's rmse: 2.6803

---------- Fold: (2 / 5) ----------

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.58326	valid's rmse: 2.38065
Early stopping, best iteration is:
[72]	train's rmse: 2.63254	valid's rmse: 2.37629

---------- Fold: (3 / 5) ----------

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.48734	valid's rmse: 2.27369
[200]	train's rmse: 2.41731	valid's rmse: 2.26185
[300]	train's rmse: 2.36948	valid's rmse: 2.25155
[400]	train's rmse: 2.33358	valid's rmse: 2.24846
Early stopping, best iteration is:
[437]	train's rmse: 2.32289	valid's rmse: 2.24614

---------- Fold: (4 / 5) ----------

Training until validation scores don't improve for 50 rounds
[100]	train's rmse: 2.4316	valid's rmse: 2.32449
[20

MemoryError: 

In [23]:
X_train.head(10)

,item_id,dept_id,cat_id,store_id,state_id,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,shift_t28,shift_t29,shift_t30,rolling_mean_t7,rolling_std_t7,rolling_mean_t30,rolling_mean_t90,rolling_mean_t180,rolling_std_t30,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,rolling_skew_t30,rolling_kurt_t30,year,month,week,day,dayofweek
0,2506,5,2,9,2,13,1,1,1,0,0,0,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
1,2507,5,2,9,2,13,1,1,1,0,0,0,2.470703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
2,2508,5,2,9,2,13,1,1,1,0,0,0,1.980469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
3,2509,5,2,9,2,13,1,1,1,0,0,0,0.979980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
4,2510,5,2,9,2,13,1,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
5,2511,5,2,9,2,13,1,1,1,0,0,0,4.878906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
6,2512,5,2,9,2,13,1,1,1,0,0,0,5.480469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
7,2513,5,2,9,2,13,1,1,1,0,0,0,0.970215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
8,2514,5,2,9,2,13,1,1,1,0,0,0,1.839844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
9,2515,5,2,9,2,13,1,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,7,29,18,3
